# Notebook 04: Conception et Exécution du Workflow LangGraph

Ce notebook est dédié à l'exécution et à l'observation de notre workflow multi-agents "Cognitive Swarm", tel que défini dans `src/graph/main_workflow.py`. Nous allons soumettre une requête complexe et suivre le déroulement des opérations à travers les différents agents et outils.

**Prérequis :**
* **Environnement de Base :** Avoir exécuté le notebook `00_setup_environment.ipynb` pour configurer l'environnement Conda, les dépendances Python, et s'assurer que le fichier `.env` à la racine du projet est correctement rempli.
* **Base de Données MongoDB :**
    * `MONGO_URI` doit être correctement configuré dans `.env` et votre instance MongoDB doit être accessible. Ceci est utilisé par le checkpointer LangGraph (`MongoDBSaver`) et les outils RAG.
    * La base de données doit être peuplée avec des documents et leurs embeddings (via `01_data_ingestion_and_embedding.ipynb` ou `scripts/run_ingestion.py`). Les embeddings stockés doivent correspondre au fournisseur configuré via `DEFAULT_EMBEDDING_PROVIDER` pour que les outils RAG fonctionnent de manière optimale.
* **Configuration des Fournisseurs de Modèles (dans `.env`) :** Le workflow utilisera les fournisseurs configurés via les variables `DEFAULT_LLM_MODEL_PROVIDER` (pour les agents) et `DEFAULT_EMBEDDING_PROVIDER` (pour la RAG et l'embedding des requêtes).
    * **Pour les LLMs des Agents (`DEFAULT_LLM_MODEL_PROVIDER`) :**
        * Si réglé sur `"openai"` : `OPENAI_API_KEY` est requise.
        * Si réglé sur `"huggingface_api"` : `HUGGINGFACE_API_KEY` et `HUGGINGFACE_REPO_ID` (pour le modèle génératif) sont requis.
        * Si réglé sur `"ollama"` : Assurez-vous que `OLLAMA_BASE_URL` pointe vers votre instance Ollama en cours d'exécution, et que `OLLAMA_GENERATIVE_MODEL_NAME` est un modèle que vous avez téléchargé (`ollama pull ...`).
    * **Pour les Embeddings (`DEFAULT_EMBEDDING_PROVIDER`, utilisé par `RetrievalEngine`) :**
        * Si réglé sur `"openai"` : `OPENAI_API_KEY` est requise (pour `LlamaIndex OpenAIEmbedding`).
        * Si réglé sur `"huggingface"` (local Sentence Transformers) : Aucune clé API spécifique n'est généralement requise pour cette étape.
        * Si réglé sur `"ollama"` : Assurez-vous que `OLLAMA_BASE_URL` est correct et que `OLLAMA_EMBEDDING_MODEL_NAME` est un modèle d'embedding disponible sur votre instance Ollama.
* **(Optionnel) Weights & Biases :** Si vous souhaitez utiliser le logging des évaluations avec W&B (via `scripts/run_evaluation.py`, non directement testé dans ce notebook mais bon à savoir), `WANDB_API_KEY` doit être configurée dans `.env`.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json
import asyncio # Pour exécuter notre fonction de workflow asynchrone
import uuid    # Pour générer des thread_id uniques
from typing import Dict, Any, List # Ajout pour pretty_print_final_state

# --- Configuration du PYTHONPATH et chargement de .env ---
# NOTE IMPORTANTE SUR LE CWD (Current Working Directory) :
# La ligne suivante `project_root = Path().resolve().parent` suppose que le CWD du notebook
# est le dossier `/notebooks/`. Si vous avez configuré VS Code pour que le CWD
# soit la racine du projet (`cognitive-swarm-agents/`), alors `Path().resolve()`
# donnerait déjà la racine du projet, et vous devriez utiliser :
# project_root = Path().resolve()
# Vérifiez votre CWD avec `import os; print(os.getcwd())` ou `from pathlib import Path; print(Path().resolve())` pour confirmer.
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
# De même, si CWD est la racine du projet, dotenv_path serait `Path().resolve() / ".env"`
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}. Assurez-vous qu'il est à la racine du projet.")

from config.settings import settings
from config.logging_config import setup_logging

# Importer la fonction d'exécution du workflow principal
# graph_app_v2_1 est le graphe compilé, utile si on veut l'inspecter ou l'utiliser directement.
from src.graph.main_workflow import run_cognitive_swarm_v2_1, graph_app_v2_1 

# Configurer le logging pour le notebook
LOG_LEVEL_NOTEBOOK = "INFO" # Changer en "DEBUG" pour un maximum de détails du workflow LangGraph
setup_logging(level=LOG_LEVEL_NOTEBOOK) 
logger = logging.getLogger("nb_04_workflow_execution")

# --- Vérification des prérequis pour le Workflow (LLMs, Embeddings, MongoDB) ---
logger.info(f"--- Configuration Active pour le Workflow (depuis settings.py et .env) ---")

# 1. Pour les LLMs génératifs (utilisés par les agents du workflow via llm_factory.py)
generative_llm_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"Fournisseur LLM génératif principal pour les agents : '{generative_llm_provider}'")
if generative_llm_provider == "openai":
    if not settings.OPENAI_API_KEY:
        logger.error("ERREUR : LLM Provider est 'openai', mais OPENAI_API_KEY n'est pas configurée.")
    if not settings.DEFAULT_OPENAI_GENERATIVE_MODEL:
         logger.warning("AVERTISSEMENT : DEFAULT_OPENAI_GENERATIVE_MODEL n'est pas explicitement défini (utilisera le défaut de ChatOpenAI).")
elif generative_llm_provider == "huggingface_api":
    if not settings.HUGGINGFACE_API_KEY:
        logger.error("ERREUR : LLM Provider est 'huggingface_api', mais HUGGINGFACE_API_KEY n'est pas configurée.")
    if not settings.HUGGINGFACE_REPO_ID:
        logger.error("ERREUR : LLM Provider est 'huggingface_api', mais HUGGINGFACE_REPO_ID n'est pas configuré.")
elif generative_llm_provider == "ollama":
    if not settings.OLLAMA_BASE_URL:
        logger.error("ERREUR : LLM Provider est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")
    if not settings.OLLAMA_GENERATIVE_MODEL_NAME:
        logger.error("ERREUR : LLM Provider est 'ollama', mais OLLAMA_GENERATIVE_MODEL_NAME n'est pas configuré.")
else:
    logger.error(f"ERREUR : Fournisseur LLM génératif inconnu ou non supporté : '{generative_llm_provider}'")

# 2. Pour les modèles d'Embedding (utilisés par RetrievalEngine via les outils)
embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'Embedding (pour RAG via RetrievalEngine) : '{embedding_provider}'")
if embedding_provider == "openai":
    if not settings.OPENAI_API_KEY: # Nécessaire pour LlamaIndex OpenAIEmbedding
        logger.error("ERREUR : Embedding Provider est 'openai', mais OPENAI_API_KEY n'est pas configurée.")
    if not settings.OPENAI_EMBEDDING_MODEL_NAME:
        logger.warning("AVERTISSEMENT : OPENAI_EMBEDDING_MODEL_NAME n'est pas explicitement défini.")
elif embedding_provider == "huggingface":
    # Les embeddings HuggingFace locaux (SentenceTransformers) ne nécessitent pas de clé API.
    if not settings.HUGGINGFACE_EMBEDDING_MODEL_NAME:
        logger.error("ERREUR : Embedding Provider est 'huggingface', mais HUGGINGFACE_EMBEDDING_MODEL_NAME n'est pas configuré.")
elif embedding_provider == "ollama":
    if not settings.OLLAMA_BASE_URL: # Partagé avec le LLM génératif Ollama
        logger.error("ERREUR : Embedding Provider est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")
    if not settings.OLLAMA_EMBEDDING_MODEL_NAME:
        logger.error("ERREUR : Embedding Provider est 'ollama', mais OLLAMA_EMBEDDING_MODEL_NAME n'est pas configuré.")
else:
    logger.error(f"ERREUR : Fournisseur d'embedding inconnu ou non supporté : '{embedding_provider}'")

# 3. Pour MongoDB (utilisé par le checkpointer LangGraph et RetrievalEngine)
if not settings.MONGO_URI or "<user>" in settings.MONGO_URI: # Ajout d'un check pour les placeholders
    logger.error("ERREUR : MONGO_URI non trouvé ou semble non configuré (contient des placeholders). Le checkpointer et le RetrievalEngine (RAG) échoueront.")
else:
    logger.info(f"MongoDB URI configuré (début): {settings.MONGO_URI[:30]}...")
    logger.info(f"Base de données MongoDB: {settings.MONGO_DATABASE_NAME}")
    logger.info(f"Collection des checkpoints LangGraph: {settings.LANGGRAPH_CHECKPOINTS_COLLECTION}")

logger.info("--- Fin de la Vérification de Configuration Active ---")

# --- Petite fonction pour afficher l'état final de manière plus structurée (inchangée) ---
# (La définition de pretty_print_final_state que vous aviez était bonne, je la garde telle quelle)
def pretty_print_final_state(final_state: Dict[str, Any]): # S'assurer que Dict et Any sont importés de typing
    print("\n--- État Final Détaillé du Graphe ---")
    if not final_state:
        print("Aucun état final retourné.")
        return
            
    for key, value in final_state.items():
        if key == "messages":
            print(f"\n  {key.upper()}:")
            if isinstance(value, list):
                for i, msg in enumerate(value[-5:]): # Afficher les 5 derniers messages pour concision
                    msg_type = getattr(msg, 'type', 'UNKNOWN_MSG_TYPE').upper()
                    msg_name = getattr(msg, 'name', None)
                    msg_content_str = str(getattr(msg, 'content', 'N/A'))
                    display_name = f"{msg_type} ({msg_name})" if msg_name else msg_type
                    
                    print(f"    Message {len(value) - 5 + i +1 if len(value)>5 else i+1}: [{display_name}]")
                    if hasattr(msg, 'tool_calls') and msg.tool_calls:
                        print(f"      Contenu: {msg_content_str[:100]}... [Appels d'outils: {len(msg.tool_calls)}]")
                        for tc in msg.tool_calls:
                            # La structure de tc peut varier, adaptez au besoin si ce n'est pas un dict direct
                            tc_args = tc.get('args') if isinstance(tc, dict) else getattr(tc, 'args', {})
                            tc_name = tc.get('name') if isinstance(tc, dict) else getattr(tc, 'name', 'unknown_tool')
                            tc_id = tc.get('id') if isinstance(tc, dict) else getattr(tc, 'id', None)
                            print(f"        Tool Call ID: {tc_id}, Name: {tc_name}, Args: {tc_args}")
                    elif msg_type == "TOOL": # ToolMessage
                        tool_call_id = getattr(msg, 'tool_call_id', 'N/A')
                        parsed_tool_content = None
                        if isinstance(msg_content_str, str):
                            try:
                                parsed_tool_content = json.loads(msg_content_str)
                            except json.JSONDecodeError:
                                pass 
                        
                        if parsed_tool_content and isinstance(parsed_tool_content, list) and parsed_tool_content:
                            print(f"      Tool Call ID: {tool_call_id} - Résultat Outil (Liste de {len(parsed_tool_content)} éléments):")
                            for item_idx, item_data in enumerate(parsed_tool_content[:2]): 
                                if isinstance(item_data, dict):
                                    print(f"        Item {item_idx+1}: { {k: str(v)[:70] + '...' if isinstance(v,str) and len(str(v)) > 70 else v for k,v in item_data.items()} }")
                                else:
                                    print(f"        Item {item_idx+1}: {str(item_data)[:100]}...")
                            if len(parsed_tool_content) > 2:
                                print("        ...")
                        else: # Si ce n'est pas une liste parsable ou si elle est vide
                            print(f"      Tool Call ID: {tool_call_id} - Contenu (Résultat Outil): {msg_content_str[:200]}...")
                    else:
                        print(f"      Contenu: {msg_content_str[:200]}...")
            else: # Si 'messages' n'est pas une liste
                print(f"    {str(value)[:500]}...")
        elif key in ["research_plan", "synthesis_output", "document_analysis_summary", "user_query", "error_message"]:
            print(f"\n  {key.upper()}:\n{str(value)[:1000]}{'...' if value and len(str(value)) > 1000 else ''}\n")
        else: 
            print(f"  {key.upper()}: {str(value)[:500]}{'...' if value and len(str(value)) > 500 else ''}")
    print("------------------------------------")

### 1. Définition d'une Requête Utilisateur Complexe

Nous allons utiliser une requête qui nécessite potentiellement plusieurs étapes de la part de nos agents (planification, recherche, analyse, synthèse).

In [ ]:
user_query = "Analyze the latest advancements in reinforcement learning for robotic locomotion, focusing on how bipedal robots achieve stable gait. Include key challenges and future research directions based on recent ArXiv papers."
# user_query = "What are common methods for robot arm path planning based on recent ArXiv papers?"

logger.info(f"Requête utilisateur pour ce test : '{user_query}'")

### 2. Exécution du Workflow "Cognitive Swarm" (Premier Passage)

Nous exécutons ici la fonction `run_cognitive_swarm_v2_1` avec notre requête. Un nouvel ID de thread (`thread_id`) sera généré.
La sortie de `astream_events` dans `run_cognitive_swarm_v2_1` affichera le flux en temps réel (chunks de LLM, appels d'outils).
Le checkpointer MongoDB (activé par défaut dans `main_workflow.py`) sauvegardera l'état à chaque étape.

In [ ]:
# Générer un ID de thread unique pour cette exécution
test_thread_id = "nb_workflow_run_" + str(uuid.uuid4())

# LOG_LEVEL_NOTEBOOK est défini dans la première cellule de ce notebook (ID 58690d91)
print(f"Lancement du workflow pour la requête avec thread_id: {test_thread_id}")
print(f"Utilisation du LLM provider configuré: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' et du provider d'embedding: '{settings.DEFAULT_EMBEDDING_PROVIDER}'.")
print(f"Les logs du workflow (niveau '{LOG_LEVEL_NOTEBOOK}') et les sorties des agents/outils via astream_events apparaîtront ci-dessous.")
print("Le traitement peut prendre plusieurs minutes en fonction de la complexité de la requête et des modèles LLM utilisés.")

# MODIFICATION : Activer nest_asyncio pour permettre asyncio.run() dans Jupyter
import nest_asyncio
nest_asyncio.apply()

final_state_run1 = None

if not settings.MONGO_URI or ("<user>" in settings.MONGO_URI and "<password>" in settings.MONGO_URI) or "<cluster_url>" in settings.MONGO_URI:
    print("\nERREUR CRITIQUE: MONGO_URI n'est pas configuré correctement dans le fichier .env (il manque ou contient des placeholders).")
    print("L'exécution du workflow est annulée car le checkpointer MongoDB est requis.")
    logger.error("MONGO_URI non configuré ou contient des placeholders. Workflow non exécuté.")
else:
    try:
        # asyncio.run() devrait maintenant fonctionner correctement grâce à nest_asyncio.apply()
        final_state_run1 = asyncio.run(run_cognitive_swarm_v2_1(user_query, thread_id=test_thread_id))
        
    except ValueError as ve: 
        logger.error(f"Erreur de configuration (ValueError) lors de l'exécution de run_cognitive_swarm_v2_1: {ve}", exc_info=True)
        print(f"\nERREUR DE CONFIGURATION PENDANT L'EXÉCUTION DU WORKFLOW : {ve}")
        print("Veuillez vérifier les configurations pour DEFAULT_LLM_MODEL_PROVIDER, DEFAULT_EMBEDDING_PROVIDER, ")
        print("et leurs dépendances respectives (clés API, URLs de base, noms de modèles exacts) dans votre fichier .env et settings.py.")
        print(f"Provider LLM actuel: {settings.DEFAULT_LLM_MODEL_PROVIDER}, Provider Embedding actuel: {settings.DEFAULT_EMBEDDING_PROVIDER}")
    except RuntimeError as re: # Attraper spécifiquement le RuntimeError si nest_asyncio n'a pas fonctionné
        if "cannot be called from a running event loop" in str(re):
            logger.error(f"Erreur RuntimeError avec asyncio.run(): {re}. 'nest_asyncio.apply()' n'a peut-être pas fonctionné comme prévu.", exc_info=True)
            print(f"\nERREUR asyncio : {re}. Assurez-vous que 'nest_asyncio' est installé et que 'nest_asyncio.apply()' a été appelé.")
        else:
            logger.error(f"Erreur RuntimeError inattendue lors de l'exécution de asyncio.run(run_cognitive_swarm_v2_1): {re}", exc_info=True)
            print(f"\nERREUR RUNTIME INATTENDUE PENDANT L'EXÉCUTION DU WORKFLOW : {re}")
    except Exception as e: 
        logger.error(f"Erreur inattendue lors de l'exécution de asyncio.run(run_cognitive_swarm_v2_1): {e}", exc_info=True)
        print(f"\nERREUR INATTENDUE PENDANT L'EXÉCUTION DU WORKFLOW : {e}")

if final_state_run1:
    # pretty_print_final_state est défini dans la première cellule de code de ce notebook (ID 58690d91)
    pretty_print_final_state(final_state_run1)
else:
    print("\nL'exécution du workflow n'a pas retourné d'état final ou a échoué avant de pouvoir retourner un état.")
    if not settings.MONGO_URI or ("<user>" in settings.MONGO_URI and "<password>" in settings.MONGO_URI) or "<cluster_url>" in settings.MONGO_URI :
        print("Rappel : MONGO_URI n'était pas (ou mal) configuré, ce qui a pu empêcher l'exécution.")

### 3. Analyse des Sorties et du Comportement

Après l'exécution :
* Examinez les logs produits dans la console du notebook (si le niveau de log est DEBUG pour `main_workflow` ou les agents, vous verrez beaucoup de détails).
* Observez la sortie finale (`synthesis_output`) dans l'état final.
* Si vous avez accès à MongoDB (par exemple, via MongoDB Compass ou un autre client), vous pouvez inspecter la collection des checkpoints (par défaut `langgraph_checkpoints` dans la base `cognitive_swarm_db`). Vous devriez y trouver des documents correspondant au `thread_id` utilisé. Chaque document représente un état sauvegardé du graphe.

### 4. (Optionnel) Exécution d'une Requête de Suivi sur le Même Thread

Si le checkpointer a fonctionné, l'historique des messages et l'état du thread précédent sont sauvegardés. Envoyer une nouvelle requête avec le *même `thread_id`* permettra au système de potentiellement utiliser cet historique.

Notre workflow actuel est plutôt linéaire et ne gère pas explicitement les "questions de suivi" pour modifier un rapport existant. Une nouvelle invocation avec le même `thread_id` ajoutera à l'historique des messages et relancera le flux depuis le début, mais les agents verront l'historique complet.

Pour une vraie "reprise" d'un graphe interrompu, LangGraph le gère automatiquement si vous relancez avec la même configuration (thread_id).

In [ ]:
# --- Test d'une Requête de Suivi (Optionnel) ---
# Décommentez les lignes ci-dessous pour tester une requête de suivi sur le même thread.
# Note: Cela relancera le flux avec l'historique des messages accumulé. Le workflow actuel
# est plutôt linéaire et une nouvelle invocation relancera le processus depuis le début,
# mais les agents verront l'historique complet. La persistance via checkpointer est démontrée.

# # S'assurer que test_thread_id est défini par la cellule précédente (ID 8ad390ec)
# if 'test_thread_id' in locals() and test_thread_id:
# #     follow_up_query = "Could you elaborate on the sim-to-real transfer challenges mentioned previously?" # Exemple de requête de suivi
# #     logger.info(f"Requête de suivi pour le thread {test_thread_id}: '{follow_up_query}'")
# #     print(f"\nLancement d'une requête de suivi sur le même thread_id: {test_thread_id}")
# #     print(f"Utilisation du LLM provider configuré: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' et du provider d'embedding: '{settings.DEFAULT_EMBEDDING_PROVIDER}'.")
# #     print("Le traitement peut prendre plusieurs minutes...")

# #     final_state_run2 = None
# #     # Vérification principale: MongoDB est essentiel.
# #     if not settings.MONGO_URI or ("<user>" in settings.MONGO_URI and "<password>" in settings.MONGO_URI) or "<cluster_url>" in settings.MONGO_URI:
# #         print("ERREUR CRITIQUE: MONGO_URI n'est pas configuré correctement. Requête de suivi annulée.")
# #         logger.error("MONGO_URI non configuré ou contient des placeholders. Requête de suivi non exécutée.")
# #     else:
# #         try:
# #             final_state_run2 = asyncio.run(run_cognitive_swarm_v2_1(follow_up_query, thread_id=test_thread_id))
# #         except ValueError as ve: 
# #             logger.error(f"Erreur de configuration (ValueError) lors de la requête de suivi: {ve}", exc_info=True)
# #             print(f"\nERREUR DE CONFIGURATION PENDANT LA REQUÊTE DE SUIVI : {ve}")
# #             print("Veuillez vérifier les configurations pour DEFAULT_LLM_MODEL_PROVIDER, DEFAULT_EMBEDDING_PROVIDER, et leurs dépendances.")
# #         except Exception as e: 
# #             logger.error(f"Erreur inattendue lors de l'exécution de la requête de suivi : {e}", exc_info=True)
# #             print(f"\nERREUR INATTENDUE PENDANT LA REQUÊTE DE SUIVI : {e}")

# #     # pretty_print_final_state est défini dans la première cellule de code de ce notebook (ID 58690d91)
# #     if final_state_run2:
# #         pretty_print_final_state(final_state_run2)
# #     else:
# #         print("\nL'exécution de la requête de suivi n'a pas retourné d'état final ou a échoué.")
# else:
# #     # Ce message s'affichera si cette cellule est exécutée avant que test_thread_id ne soit défini.
# #     print("\nVariable 'test_thread_id' non trouvée ou non initialisée.")
# #     print("Veuillez exécuter la cellule précédente (exécution du workflow principal) pour définir un 'test_thread_id' avant de décommenter et d'exécuter cette cellule.")
# #     logger.warning("'test_thread_id' non défini. Saut de la cellule de requête de suivi optionnelle.")

### 5. Inspection des Checkpoints dans MongoDB (Conceptuel)

Si le `MongoDBSaver` est actif, vous pouvez vous connecter à votre instance MongoDB et examiner la collection `langgraph_checkpoints` (ou le nom que vous avez configuré dans `settings.py`). Vous y trouverez des documents JSON représentant les différents états sauvegardés pour chaque `thread_id`.

Chaque document de checkpoint contient typiquement :
* `thread_id`
* `thread_ts` (un timestamp identifiant ce snapshot spécifique de l'état)
* `checkpoint` (l'état sérialisé du graphe, incluant les valeurs des canaux comme `messages`)
* `metadata` (métadonnées associées au checkpoint)
* `parent_ts` (s'il y a un checkpoint parent)

Cela démontre la persistance et la capacité de reprise du workflow.

## Conclusion de la Démonstration du Workflow

Ce notebook a permis de lancer le workflow LangGraph complet et d'observer son exécution.
Les prochaines étapes pourraient inclure :
* Des tests avec des requêtes plus variées.
* L'analyse détaillée des checkpoints dans MongoDB.
* L'utilisation du script `scripts/run_evaluation.py` pour évaluer quantitativement les résultats.
* L'amélioration itérative de la logique de routage et des prompts des agents dans `src/graph/main_workflow.py` et `src/agents/agent_architectures.py`.